### Quelle der Daten: 
https://www.bundestag.de/services/opendata

### Erläuterungen
WP steht für Wahlperiode

In [1]:
#TODO: this is not the best way 
import sys
sys.path.append("../src")

In [2]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET
from typing import Dict, List
from collections import defaultdict
from config import LIST_OF_COLORS

NameError: name 'os' is not defined

In [ ]:
sns.set_theme()

In [ ]:
# Pfad zum Hauptverzeichnis des Projekts
PROJECT_ROOT = Path(os.getcwd()).parent

DATA_PATH = PROJECT_ROOT / 'data'
PLOT_PATH = PROJECT_ROOT / 'plots'

CURRENT_YEAR = '2021'  # oder '2024' für aktuelle Daten

MDB_XML_PATH = DATA_PATH / CURRENT_YEAR / 'input' / 'MDB_STAMMDATEN.XML'

#one row per individuum, one col per wp containing 0 / 1
DF_MDB_PATH = DATA_PATH / CURRENT_YEAR / 'output' / 'df_mdb.csv'

# one row per individuum per wp, containting e.g. 12. so one abgeordneter who has been in parliament for several wp will receive one row per wp
DF_MDB_WP_PATH = DATA_PATH / CURRENT_YEAR / 'output' / 'df_mdb_wp.csv' 

# startdaten der Wahlperioden
DF_MDB_WP_STARTDATEN_PATH = DATA_PATH / CURRENT_YEAR / 'output' / 'wp_startdaten.csv'

# parse xml data into dataFrame / .csv fomat

In [ ]:
def get_mdb_data(mdb_xml_path: Path = MDB_XML_PATH) -> pd.DataFrame:
    """Parse MDB XML data and return a DataFrame."""
    tree = ET.parse(mdb_xml_path)
    root = tree.getroot()

    mdb_list = []
    for mdb in root.findall('MDB'):
        mdb_dict = defaultdict(lambda: 0)
        mdb_dict['ID'] = mdb.find('ID').text
        namen = mdb.find('NAMEN/NAME')
        mdb_dict['NACHNAME'] = namen.find('NACHNAME').text
        mdb_dict['VORNAME'] = namen.find('VORNAME').text

        bio = mdb.find('BIOGRAFISCHE_ANGABEN')
        for element in bio:
            mdb_dict[element.tag] = element.text

        wahlperioden = mdb.find('WAHLPERIODEN')
        mdb_dict['ANZ_WAHLPERIODEN'] = len(wahlperioden)
        for wp in wahlperioden:
            mdb_dict[int(wp.find('WP').text)] = 1

        mdb_list.append(dict(mdb_dict))

    return pd.DataFrame(mdb_list)

In [ ]:
def get_mdb_wp_data(df_mdb: pd.DataFrame) -> pd.DataFrame:
    """
    Reformat data from wide to long format, creating one row per person per Wahlperiode.
    """
    wps: List[int] = [col for col in df_mdb.columns if isinstance(col, int)]
    columns_to_keep: List[str] = [col for col in df_mdb.columns if not isinstance(col, int)]
    
    # Melt the dataframe to long format
    df_long = df_mdb.melt(id_vars=columns_to_keep, 
                          value_vars=wps, 
                          var_name='WP', 
                          value_name='is_active')
    
    # Filter only active Wahlperioden and drop the 'is_active' column
    return df_long[df_long['is_active'] == 1].drop('is_active', axis=1)


In [ ]:
df_mdb = get_mdb_data(MDB_XML_PATH)
print(df_mdb.shape)
df_mdb.head()

In [ ]:
df_mdb_wp = get_mdb_wp_data(df_mdb)

In [ ]:
# deleteme
df_mdb_wp[df_mdb_wp.BERUF.str.contains('jurist', na=False)][['NACHNAME', 'VORNAME']]

# compute Age

In [ ]:
df_mdb['GEBURSDATUM_DATE'] = pd.to_datetime(df_mdb.GEBURTSDATUM, format='%d/%m/%Y', errors='coerce')

In [ ]:
df_mdb.head()

In [ ]:
wp_startdaten = {
    1:datetime(1949, 9, 7), # Der 1. Deutsche Bundestag bestand zwischen dem 7. September 1949[1] und dem 7. September 1953
    2:datetime(1953, 10, 6), # Der 2. Deutsche Bundestag bestand zwischen dem 6. Oktober 1953[1] und dem 6. Oktober 1957
    3:datetime(1957, 10, 15), # Der 3. Deutsche Bundestag bestand zwischen dem 15. Oktober 1957[1] und dem 15. Oktober 1961
    4:datetime(1961, 10, 17), # Der 4. Deutsche Bundestag bestand zwischen dem 17. Oktober 1961[1] und dem 17. Oktober 196
    5:datetime(1965, 10, 19), # Der 5. Deutsche Bundestag bestand zwischen dem 19. Oktober 1965[1] und dem 19. Oktober 1969
    6:datetime(1969, 10, 20), # Der 6. Deutsche Bundestag bestand zwischen dem 20. Oktober 1969[1] und dem 13. Dezember 1972
    7:datetime(1972, 12, 13), #? Der 7. Deutsche Bundestag wurde am 19. November 1972 gewählt
    8:datetime(1976, 12, 14), # Der 8. Deutsche Bundestag bestand zwischen dem 14. Dezember 1976[1] und dem 4. November 1980
    9:datetime(1980, 11, 4),# Der 9. Deutsche Bundestag bestand zwischen dem 4. November 1980[1] und dem 29. März 1983
    10:datetime(1983, 3, 29), # Der 10. Deutsche Bundestag bestand zwischen dem 29. März 1983[1] und dem 18. Februar 1987
    11:datetime(1987, 2, 18), # Der 11. Deutsche Bundestag bestand zwischen dem 18. Februar 1987[1] und dem 20. Dezember 1990
    12:datetime(1990, 12, 20), # Der 12. Deutsche Bundestag bestand zwischen dem 20. Dezember 1990[1] und dem 10. November 1994
    13:datetime(1994, 11, 10), # Der 13. Deutsche Bundestag bestand zwischen dem 10. November 1994[1] und dem 26. Oktober 1998
    14:datetime(1998, 10, 26), # Der 14. Deutsche Bundestag bestand zwischen dem 26. Oktober 1998[1] und dem 17. Oktober 2002
    15:datetime(2002, 10, 17), # Der 15. Deutsche Bundestag bestand zwischen dem 17. Oktober 2002[1] und dem 18. Oktober 2005
    16:datetime(2005, 10, 18), # Der 16. Deutsche Bundestag bestand zwischen dem 18. Oktober 2005[1] und dem 27. Oktober 2009
    17:datetime(2009, 10, 27), # Der 17. Deutsche Bundestag bestand zwischen dem 27. Oktober 2009[1] und dem 22. Oktober 2013
    18:datetime(2013, 10, 22), # Der 18. Deutsche Bundestag bestand vom 22. Oktober 2013 bis zum 24. Oktober 2017
    19:datetime(2017, 10, 24), #  Seine konstituierende Sitzung fand am 24. Oktober 2017 statt,
    20:datetime(2021, 10, 26) #Die Wahlperiode begann mit der konstituierenden Sitzung am 26. Oktober 2021
}

df_wp_startdaten=pd.DataFrame(wp_startdaten, index=[0]).T
df_wp_startdaten.columns =[ 'START_DATE']
print(df_wp_startdaten.head())
df_wp_startdaten.to_csv(DF_MDB_WP_STARTDATEN_PATH)

## Insight: es werden immer mehr Abgeordnete

In [ ]:
# Daten vorbereiten
wps = [column for column in df_mdb.columns if isinstance(column, int)]
abgeordnete_pro_wp = df_mdb[wps].sum().sort_index()

# Plot erstellen
fig, ax = plt.subplots(figsize=(14, 8))

# Balkendiagramm
sns.barplot(x=abgeordnete_pro_wp.index, y=abgeordnete_pro_wp.values, ax=ax)

# Beschriftungen hinzufügen
ax.set_title('Anzahl der Abgeordneten pro Wahlperiode', fontsize=20)
ax.set_xlabel('Wahlperiode', fontsize=14)
ax.set_ylabel('Anzahl der Abgeordneten', fontsize=14)

# Werte über den Balken anzeigen (als ganze Zahlen)
for i, v in enumerate(abgeordnete_pro_wp.values):
    ax.text(i, v + 5, f'{int(v)}', ha='center', va='bottom')

# Y-Achse bei 0 beginnen lassen und bis zum maximalen Wert plus etwas Abstand gehen
y_max = max(abgeordnete_pro_wp.values)
ax.set_ylim(0, y_max * 1.1)  # 10% Abstand nach oben für die Beschriftungen

# Grid für bessere Lesbarkeit
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Layout optimieren
plt.tight_layout()

# SVG
plt.savefig(PLOT_PATH / 'anzahl_abgeordneter.svg', format='svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df_mdb[wps].sum().sort_index().tail()

In [ ]:
# total number of abgeordnete (counting multiple wps as multiple entries)
df_mdb[wps].sum().sum()

# look at one VITA KURZ

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(df_mdb_wp[df_mdb_wp.NACHNAME =='Baerbock'].iloc[0].VITA_KURZ)

# clean data 1. missing values

In [ ]:
dimensions_replace_na = ['NACHNAME', 'VORNAME', 'GEBURTSDATUM', 'GEBURTSORT', 'GEBURTSLAND', 'STERBEDATUM', 'GESCHLECHT', 'FAMILIENSTAND', 'RELIGION', 'BERUF', 'PARTEI_KURZ']
df_mdb[dimensions_replace_na].isna().sum()

In [ ]:
df_mdb[dimensions_replace_na] = df_mdb[dimensions_replace_na].fillna('UNBEKANNT')

# 2. replace values: PARTEI_KURZ

In [ ]:
df_mdb.PARTEI_KURZ.value_counts()[:12]

In [ ]:
# some cleaning necessary: 'BÜNDNIS 90/DIE GRÜNEN', 'DIE GRÜNEN/BÜNDNIS 90' => 'GRÜNE'
for grünen_alias in ['BÜNDNIS 90/DIE GRÜNEN', 'DIE GRÜNEN/BÜNDNIS 90', 'GRÜNE']:
    df_mdb.replace(grünen_alias, 'DIE GRÜNEN', inplace=True)

# replace values: RELIGION

In [ ]:
df_mdb['RELIGION_MAPPED'] = df_mdb['RELIGION']
df_mdb.RELIGION_MAPPED.value_counts().head(12)

In [ ]:
for katholisch_alias in ['römisch-katholisch', 'alt-katholisch']:
    df_mdb['RELIGION_MAPPED'].replace(katholisch_alias, 'katholisch', inplace=True)

In [ ]:
for evangelisch_alias in ['evangelisch-lutherisch', 'evangelisch-reformiert', 'evangelisch-freikirchlich', 'protestantisch']:
    df_mdb['RELIGION_MAPPED'].replace(evangelisch_alias, 'evangelisch', inplace=True)

In [ ]:
# it's not the same but its similar
for konfessionslos_alias in ['religionslos', 'Atheist', 'Atheistin']:
    df_mdb['RELIGION_MAPPED'].replace(konfessionslos_alias, 'konfessionslos', inplace=True)

In [ ]:
for konfessionslos_alias in ['Islam']:
    df_mdb['RELIGION_MAPPED'].replace(konfessionslos_alias, 'muslimisch', inplace=True)

In [ ]:
# erkenntnis: nur christen und einige wenige konfessionslose
df_mdb['RELIGION_MAPPED'].value_counts().plot(kind='bar')

In [ ]:
df_mdb['RELIGION_MAPPED'].value_counts().plot(kind='bar')

# replace values: FAMILIENSTAND

In [ ]:
# gehen sie weiter. hier gibt es kein Klischee zu erkennen!
df_mdb['FAMILIENSTAND_MAPPED'] = df_mdb['FAMILIENSTAND']
df_mdb.FAMILIENSTAND_MAPPED= df_mdb.FAMILIENSTAND_MAPPED.apply(lambda x: x.replace('verheiratet', 'verh.'))
df_mdb.FAMILIENSTAND_MAPPED= df_mdb.FAMILIENSTAND_MAPPED.apply(lambda x: x.replace('geschieden', 'gesch.'))

df_mdb[['ID', 'FAMILIENSTAND_MAPPED']].groupby('FAMILIENSTAND_MAPPED').count().sort_values(by='ID', ascending=False).head(28).plot(kind='bar', figsize=(12,7))

In [ ]:
indikator = re.compile('^verh.,\s([4-9] |\d{2,}) *Kind')
values_to_replace = set(df_mdb[df_mdb.FAMILIENSTAND_MAPPED.str.contains(indikator)].FAMILIENSTAND_MAPPED)
print(values_to_replace)
replace_value = 'verh., >3 Kinder'
df_mdb['FAMILIENSTAND_MAPPED'].replace(values_to_replace, replace_value, inplace=True)

In [ ]:
indikator = re.compile('^gesch.,\s([1-9] |\d{2,}) *Kind')
values_to_replace = set(df_mdb[df_mdb.FAMILIENSTAND_MAPPED.str.contains(indikator)].FAMILIENSTAND_MAPPED)
print(values_to_replace)
replace_value = 'gesch., >0 Kinder'
df_mdb['FAMILIENSTAND_MAPPED'].replace(values_to_replace, replace_value, inplace=True)

In [ ]:
df_mdb[['ID', 'FAMILIENSTAND_MAPPED']].groupby('FAMILIENSTAND_MAPPED').count().sort_values(by='ID', ascending=False).head(16)

# replace values: BERUFE

In [ ]:
# number of different jobs: 2k
len(set(df_mdb.BERUF))

In [ ]:
df_mdb['BERUF_MAPPED'] = df_mdb['BERUF'].copy()

In [ ]:
df_mdb[['ID', 'BERUF']].groupby('BERUF').count().sort_values(by='ID', ascending=False)[8:16]

In [ ]:
# deleteme
#df_mdb['len_beruf'] = df_mdb['BERUF'].apply(lambda x: len(str(x)))
#print(df_mdb.sort_values(by ='len_beruf', ascending=False)[['VORNAME', 'NACHNAME', 'BERUF']].head(3))
print(df_mdb.loc[3456][['PARTEI_KURZ', 'VORNAME', 'NACHNAME', 'BERUF']])
print(df_mdb.loc[3456].BERUF)
#Notar, Sprecher für Recht und Verbraucherschutz, Obmann im Parlamentarischen Kontrollgremium, Staatssekretär a. D

In [ ]:
sys.path.append("..") # TODO: this is not pretty!
from importlib import reload
import src.berufe_mapping as berufe_mapping
reload(berufe_mapping)

In [ ]:

'''very basic cleaning: lowercasing, removing dipl., (fh) and a.d.'''
df_mdb = berufe_mapping.basic_cleaning_berufe(df_mdb , column = 'BERUF_MAPPED')

In [ ]:
df_mdb[['ID', 'BERUF_MAPPED']].groupby('BERUF_MAPPED').count().sort_values(by='ID', ascending=False).head(8)

In [ ]:
dict_berufe={}
dict_berufe['Jurist*in'] = ['anwalt', 'jurist', 'richter', 'notar', re.compile('dr.*\s*jur.*'), 'syndikus', 'rechtsberater']
dict_berufe['Land-/Forstwirt*in'] = ['landwirt', '^[a-z]bauer\s', 'bauer', re.compile('agrar+'), 'forst']
dict_berufe['Unterehmer*in'] = ['unternehmer'] # 'geschäftsführer' passt leider nicht wegen z.B. Parl. Geschäftssführer
dict_berufe['Ingenieur*in'] = ['ingenieur', 'maschinenbau', 'architekt']
dict_berufe['Journalist*in'] = ['journalist', 'redakteur', 'publizist', 'schriftsteller']
dict_berufe['Verleger*in'] = ['verleger', 'verlags']

# direktor: nö, sonst bezirksdirektor museumsdirektor etc
dict_berufe['Lehrer*in'] = ['erzieher', 'pädagog', 'lehrer', 'studienrat', 'studiendirektor', 'schulrat',
                            'grundschul', 'hauptshul', 'sonderschul', 'waldorf', 'realschul', 'gymnasi',
                            'volkshochschu', 'berufsschul', 'fremdsprachen',
                           'schul'] #evtl trennen Erzieher - Lehrer
dict_berufe['Professor*in'] = ['dozent', 'professor', 'prof.', 'hochschull', 'hochschulpr']
dict_berufe['Kaufmann/-frau'] = ['kaufm']
dict_berufe['Volkswirt*in'] = ['volkswirt']
dict_berufe['Berufspolitiker*in'] = ['regierungsangestellt', 'stadtamtmann', 'stadtoberinspektor', 'Landesgeschäftsführer',
                                    'landr(at|ätin)', re.compile('ministerialr(a|ä)t'), 'staatssekret', 'bürgermeist', 
                                     'regierungsrat', re.compile('regierungs(vize)*präs'), 'regierung',
                                     'stadtdirektor', 'ministerialdirektor', 'regierungsdirektor', 'gemeindedirektor', 'regierungsdirektor',
                                     'minister', 'bundeskanz', 'bundestagsp', re.compile('präsident(in)* d\.*b\.*t\.*')]
dict_berufe['Arzt/Ärztin'] = ['arzt', 'psycholog', 'psychother', 'apotheker']
dict_berufe['Theolog*in'] = ['pfarrer', 'theolog', 'diakon']
dict_berufe['Betriebswirt*in'] = ['betriebswirt', 'verwaltungs', 'steuerberater', 'bankdirektor']
dict_berufe['Wirtschaftswissenschaftler*in'] = ['wirtschaftsw', 'ökonom', 'prokurist']
dict_berufe['Geisteswissenschaftler*in'] = ['politolog', 'politikwiss', 'historik', 'philosoph', 'philolog', 'soziolog', 'sozialwissensch', 'kulturwissenschaft']
dict_berufe['Naturwissenschaftler*in'] = ['chemik', 'chemie', 'physik', 'geophysik', 'biolog', 'mathemat', 'informat']

dict_berufe['Handwerker*in'] = ['elektro', 'fahrzeug', 'handwerk', 'mechanik',
                               'schlosser', 'maurer', 'beton', 'maler', 'lackier', 'tischler', 'schreiner',
                               'bäcker', 'konditor' 'koch', 'köchin', 'müller', 'bergmann', 'werkzeugmacher']
dict_berufe['Militär'] = ['leutnant', 'oberst^u', 'soldat', re.compile('general\s')] # not oberstudienrat ;)
dict_berufe['Beamter'] = ['beamter']# problematisch: Berufspolitiker, Lehrer, Militär sind auch beamte

In [ ]:
def replace_berufsklasse(berufsklasse = 'Jurist*in', dict_berufe = dict_berufe, df = df_mdb):
    values_to_replace = set()
    # TODO without loop but use 'any'
    for indikator in dict_berufe[berufsklasse]:
        values_to_replace = values_to_replace.union(set(df_mdb[df_mdb.BERUF_MAPPED.str.contains(indikator)].BERUF_MAPPED))

    print(values_to_replace)
    df_mdb.replace(list(values_to_replace), berufsklasse, inplace=True)
    return(df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Jurist*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Land-/Forstwirt*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Unterehmer*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Ingenieur*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
len(set(df_mdb.BERUF_MAPPED))

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Journalist*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Lehrer*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Kaufmann/-frau', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Professor*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
len(set(df_mdb.BERUF_MAPPED))

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Volkswirt*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Berufspolitiker*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Arzt/Ärztin', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Theolog*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Betriebswirt*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Wirtschaftswissenschaftler*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Geisteswissenschaftler*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Naturwissenschaftler*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Verleger*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Handwerker*in', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Militär', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
df_mdb = replace_berufsklasse(berufsklasse = 'Beamter', dict_berufe = dict_berufe, df = df_mdb)

In [ ]:
len(set(df_mdb.BERUF_MAPPED))

In [ ]:
df_mdb[['ID', 'BERUF_MAPPED']].groupby('BERUF_MAPPED').count().sort_values(by='ID', ascending=False)[16:32]

In [ ]:
plt.figure(figsize=(16,8))
df_mdb.groupby('BERUF_MAPPED')['ID'].count().sort_values(ascending=False)[:32].plot(kind='bar')

In [ ]:
len(set(df_mdb['BERUF_MAPPED']))

In [ ]:
plt.figure(figsize=(16,8))
df_mdb.groupby('BERUF_MAPPED')['ID'].count().sort_values(ascending=False).tail(33)#.plot(kind='bar')

# construct df_mdb_wp

In [ ]:
df_mdb_wp = get_mdb_wp_data(df_mdb)
print(df_mdb_wp.shape)

In [ ]:
df_mdb_wp[df_mdb_wp.NACHNAME == 'Merkel'].head()

# compute age

In [ ]:
df_wp_startdaten = pd.read_csv(DF_MDB_WP_STARTDATEN_PATH, index_col=0)
df_wp_startdaten

In [ ]:
df_wp_startdaten['START_DATE'] = pd.to_datetime(df_wp_startdaten['START_DATE'])

In [ ]:
df_mdb_wp = df_mdb_wp.join(df_wp_startdaten, on='WP')

In [ ]:
df_mdb_wp['START_DATE'] = pd.to_datetime(df_mdb_wp['START_DATE'])
df_mdb_wp['GEBURTSDATUM'] = pd.to_datetime(df_mdb_wp['GEBURTSDATUM'])

In [ ]:
def match_age(x):
    if x < 30:
        return ('< 30')
    elif x < 40:
        return('30 - 40')
    elif x < 50:
        return('40 - 50')
    elif x < 60:
        return('50 - 60')
    elif x < 70:
        return('70 - 80')
    else:
        return('>= 80')

In [ ]:
match_age(80)

In [ ]:
df_mdb_wp['START_AGE_IN_YEARS'] = df_mdb_wp['START_DATE'] - df_mdb_wp['GEBURTSDATUM']
df_mdb_wp['START_AGE_IN_YEARS'] = df_mdb_wp['START_AGE_IN_YEARS']/np.timedelta64(1,'Y')

df_mdb_wp

In [ ]:
df_mdb_wp['START_AGE_IN_YEARS'].hist(bins=32)

In [ ]:
#df_mdb_wp['START_AGE_IN_YEARS'].plot(kind='bar')
df_mdb_wp['START_AGE_IN_YEARS'][:4]

In [ ]:
df_mdb_wp['START_AGE_IN_YEARS_MAPPED'] = df_mdb_wp['START_AGE_IN_YEARS'].apply(lambda x: match_age(x)) 
df_mdb_wp['START_AGE_IN_YEARS_MAPPED'].hist()

In [ ]:
list_of_altersklassen = sorted(list(set(df_mdb_wp['START_AGE_IN_YEARS_MAPPED'])))
list_of_altersklassen = ['< 30', '30 - 40', '40 - 50', '50 - 60', '70 - 80',  '>= 80']
list_of_altersklassen

# Jahre im Amt

In [ ]:
df_mdb_wp = df_mdb_wp.join(df_mdb_wp[['ID', 'START_DATE']].groupby('ID').min(), how='left', rsuffix='_FIRST', on='ID')

In [ ]:
df_mdb_wp['NUM_YEARS_IN_BT'] = df_mdb_wp['START_DATE'] - df_mdb_wp['START_DATE_FIRST']
df_mdb_wp['NUM_YEARS_IN_BT'] = df_mdb_wp['NUM_YEARS_IN_BT']/np.timedelta64(1,'Y')

In [ ]:
df_mdb_wp.shape

In [ ]:
df_mdb_wp.columns

In [ ]:
# delemte
selected_df = df_mdb_wp[(df_mdb_wp['WP']>= 1) & (df_mdb_wp['WP']<= 19)]
#selected_df = selected_df[selected_df['PARTEI_KURZ'].isin(selected_parteien)]
grouped = selected_df[['ID', 'START_DATE', 'NUM_YEARS_IN_BT', 'PARTEI_KURZ']].groupby(['START_DATE', 'PARTEI_KURZ']).mean()

In [ ]:
grouped.reset_index(inplace=True)

In [ ]:
# delemte
grouped[grouped['PARTEI_KURZ']=='CDU']['NUM_YEARS_IN_BT'].plot()

In [ ]:
# deleteme 
grouped[grouped['START_DATE']=='2017-10-24']
# START_DATE

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

selected_parties=['SPD', 'CDU', 'FDP', 'CSU']
selected_df = df_mdb_wp[(df_mdb_wp['WP']>= 1) & (df_mdb_wp['WP']<= 19)]
selected_df = selected_df[selected_df['PARTEI_KURZ'].isin(selected_parties)]
grouped = selected_df[['ID', 'START_DATE', 'NUM_YEARS_IN_BT', 'PARTEI_KURZ']].groupby(['START_DATE', 'PARTEI_KURZ']).mean()
#grouped.reset_index(inplace=True)
grouped.head()

In [ ]:
#new_index = pd.MultiIndex.from_product([selected_parties, sorted(set(grouped.reset_index()['WP']))], names=['PARTEI_KURZ', 'WP'])
new_index = pd.MultiIndex.from_product([grouped.index.levels[0], grouped.index.levels[1]], names=['START_DATE', 'PARTEI_KURZ'])
new_index#

In [ ]:
    # create entries also for 0 values
grouped_reindexed = grouped.reindex(new_index, fill_value=0)
grouped_reindexed.head(12)

In [ ]:
# double deleteme
df_mdb_wp.sort_values(by='NUM_YEARS_IN_BT')[['NACHNAME', 'VORNAME', 'PARTEI_KURZ', 'WP', 'NUM_YEARS_IN_BT']].tail()

In [ ]:
df_mdb_wp[df_mdb_wp.PARTEI_KURZ=='FDP'].sort_values(by='NUM_YEARS_IN_BT')[['NACHNAME', 'VORNAME', 'PARTEI_KURZ', 'WP', 'NUM_YEARS_IN_BT']].tail()

In [ ]:
grouped_reindexed.reset_index(inplace=True)

In [ ]:
# make it orderable
grouped_reindexed['PARTEI_KURZ'] = pd.Categorical(grouped_reindexed['PARTEI_KURZ'], selected_parties)

In [ ]:
grouped_reindexed.sort_values(by=['START_DATE', 'PARTEI_KURZ'], inplace=True)

In [ ]:
#fig = px.scatter(x=grouped['NUM_YEARS_IN_BT'].index, y= grouped['NUM_YEARS_IN_BT'])
fig = go.Figure(data=px.line(grouped_reindexed, x='START_DATE', y= 'NUM_YEARS_IN_BT', 
                                color='PARTEI_KURZ', color_discrete_sequence = LIST_OF_COLORS))
fig.update_layout(title='Bleibedauer der Abgeordneten im Bundestag',
               xaxis_title='',
               yaxis_title='Jahre im BT bei Beginn der WP')
fig.show()

In [ ]:
a

In [ ]:
# end deleteme

# save as csv

In [ ]:
df_mdb.to_csv(DF_MDB_PATH, index=None)
df_mdb_wp.to_csv(DF_MDB_WP_PATH, index=None)

# read in data. exploratory data analysis

In [ ]:
df_mdb = pd.read_csv(DF_MDB_PATH, index_col=None)
df_mdb_wp = pd.read_csv(DF_MDB_WP_PATH, index_col=None)

In [ ]:
df_mdb.columns

In [ ]:
df_mdb.tail()

In [ ]:
df_mdb_wp[['ID', 'WP']].groupby('WP').count().sort_values(by='WP', ascending=False).head()

# quality checks

In [ ]:
df_mdb_wp[(df_mdb_wp.WP==19) & (df_mdb_wp.PARTEI_KURZ == 'CDU')]

In [ ]:
df_mdb_wp[(df_mdb_wp.WP==18) & (df_mdb_wp.PARTEI_KURZ == 'CDU') & (df_mdb_wp.BERUF.str.contains('Bürgermeister'))]